In [ ]:
import os
import pathlib
import keras
import numpy as np
import tensorflow as tf

In [ ]:
root_dir = pathlib.Path('../data/Aksara-Lampung')

In [ ]:
def CreateDataset(data_dir, type=None):
    data_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        label_mode='categorical',
        color_mode='rgb',
        batch_size=32,
        image_size=(150, 150),
        seed=123,
        validation_split=0.2,
        subset=type
    )
    
    return data_ds

train_ds = CreateDataset(root_dir, 'training')
valid_ds = CreateDataset(root_dir, 'validation')

In [ ]:
class_names = train_ds.class_names
print('Class Names:\n {}'.format(class_names))

In [ ]:
import matplotlib.pyplot as plt

def PlotDataset(dataset, class_list):
    plt.figure(figsize=(16, 16))
    for images, labels in dataset.take(1):
        for i in range(16):
            plt.subplot(4, 4, i + 1)
            plt.imshow(images[i].numpy().astype('uint8'))
            class_index = np.argmax(labels[i])
            plt.title(class_list[class_index])
            plt.axis('off')

PlotDataset(train_ds, class_names)

In [ ]:
for image_batch, labels_batch in train_ds:
  print('Batch Image Shape:', image_batch.shape)
  print('Batch Labels Shape:', labels_batch.shape)
  break

In [ ]:
# DON'T RUN THIS IF USING TRANSFER LEARNING

RESCALE = tf.keras.models.Sequential([keras.layers.Rescaling(1./255.)])

train_ds = train_ds.map(lambda x, y: (RESCALE(x), y))
valid_ds = valid_ds.map(lambda x, y: (RESCALE(x), y))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
inputs = tf.keras.Input(shape=(150, 150, 3))

model = tf.keras.models.Sequential([
    inputs, 
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(700, activation='relu'),
    tf.keras.layers.Dense(560, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(28, activation='softmax', name='classification_layers')
])

model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('val_accuracy')>0.99 and logs.get('accuracy')>0.99:
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
history = model.fit(
    train_ds,
    epochs=50,
    validation_data=valid_ds,
    verbose=1,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5), callbacks]
)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
def save_my_model(model, history, type=None):
    accuracy = history.history['accuracy'][-1] * 1000
    val_accuracy = history.history['val_accuracy'][-1] * 1000
    
    model.save(f'../models/model_bali_{type}_{int(accuracy)}_{int(val_accuracy)}.h5')

In [ ]:
save_my_model(model, history, type='conv')

# Using Transfer Learning

In [ ]:
base_model = tf.keras.applications.EfficientNetV2B0(
    include_top=False,
    weights='imagenet',
    input_shape=(150, 150, 3)
)

base_model.trainable = False

inputs = tf.keras.Input(shape=(150, 150, 3))

model = tf.keras.models.Sequential([
    inputs,
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(716, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='softmax')
], name='Lampungnese_Script')

model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=50,
    validation_data=valid_ds,
    verbose=1,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3), callbacks]
)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.ylim(bottom=0.80)
plt.legend(['Train', 'Validation'], loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
save_my_model(model, history, type='efficientnet_v2')